# CIT 5450 Final Project
Joanne Crean, Juan Goleniowski, Joseph Ricci

Before running, be sure to create a new Kaggle API token and save it as 'kaggle.json' in your default Google Drive location (/content/drive/MyDrive/)

In [57]:
# Installs
!pip install -q kaggle
!pip install pandasql
!pip install sqlalchemy==1.4.46

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [58]:
# Imports
import pandas as pd
import pandasql as ps
from google.colab import drive
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [59]:
# Setup
drive.mount('/content/drive')
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [60]:
# Download Datasets
# Spotify Top 200 and Viral 50 Charts for 2017 through 2021 Dataset
!!kaggle datasets download -d dhruvildave/spotify-charts

# Spotify Audio Features Datasets
!!kaggle datasets download -d rodolfofigueroa/spotify-12m-songs
!!kaggle datasets download -d muhmores/spotify-top-100-songs-of-20152019
!!kaggle datasets download -d sashankpillai/spotify-top-200-charts-20202021
!!kaggle datasets download -d maharshipandya/-spotify-tracks-dataset
!!kaggle datasets download -d vatsalmavani/spotify-dataset
!!kaggle datasets download -d nandhakumarss/spotify-song-tracks
!!kaggle datasets download -d elemento/music-albums-popularity-prediction

# TikTok Popular Songs Dataset
!!kaggle datasets download -d sveta151/tiktok-popular-songs-2019
!!kaggle datasets download -d sveta151/tiktok-popular-songs-2020
!!kaggle datasets download -d sveta151/tiktok-popular-songs-2021

['Downloading tiktok-popular-songs-2021.zip to /content',
 '',
 '  0% 0.00/11.0k [00:00<?, ?B/s]',
 '',
 '100% 11.0k/11.0k [00:00<00:00, 6.49MB/s]']

In [61]:
# Unzip Datasets
!unzip /content/spotify-charts.zip
!unzip /content/spotify-12m-songs.zip
!unzip /content/spotify-top-100-songs-of-20152019.zip
!unzip /content/spotify-top-200-charts-20202021.zip
!unzip /content/-spotify-tracks-dataset.zip
!unzip /content/spotify-dataset.zip
!unzip /content/spotify-song-tracks.zip
!unzip /content/music-albums-popularity-prediction.zip
!unzip /content/tiktok-popular-songs-2019
!unzip /content/tiktok-popular-songs-2020
!unzip /content/tiktok-popular-songs-2021

Archive:  /content/spotify-charts.zip
  inflating: charts.csv              
Archive:  /content/spotify-12m-songs.zip
  inflating: tracks_features.csv     
Archive:  /content/spotify-top-100-songs-of-20152019.zip
  inflating: Spotify 2010 - 2019 Top 100 Songs.xlsx  
  inflating: Spotify 2010 - 2019 Top 100.csv  
Archive:  /content/spotify-top-200-charts-20202021.zip
  inflating: spotify_dataset.csv     
Archive:  /content/-spotify-tracks-dataset.zip
  inflating: dataset.csv             
Archive:  /content/spotify-dataset.zip
  inflating: data/data.csv           
  inflating: data/data_by_artist.csv  
  inflating: data/data_by_genres.csv  
  inflating: data/data_by_year.csv   
  inflating: data/data_w_genres.csv  
Archive:  /content/spotify-song-tracks.zip
  inflating: SpotifyFeatures.csv     
Archive:  /content/music-albums-popularity-prediction.zip
  inflating: sample_solution.csv     
  inflating: test.csv                
  inflating: train.csv               
Archive:  /content/tiktok

In [62]:
# Clean up directory to save space
!rm sample_data/*
!rm -d sample_data
!rm ./*.zip
!rm sample_solution.csv
!rm ./*.xlsx

rm: cannot remove 'sample_data/*': No such file or directory
rm: cannot remove 'sample_data': No such file or directory


In [63]:
# Read the csv files and save them to pandas dataframes
df_charts = pd.read_csv('charts.csv')
df_song_features_1 = pd.read_csv('tracks_features.csv')
df_song_features_2 = pd.read_csv('Spotify 2010 - 2019 Top 100.csv')
df_song_features_3 = pd.read_csv('spotify_dataset.csv')
df_song_features_4 = pd.read_csv('dataset.csv')
df_song_features_5 = pd.read_csv('data/data.csv')
df_song_features_6 = pd.read_csv('SpotifyFeatures.csv')
df_song_features_7 = pd.read_csv('train.csv')
df_song_features_10 = pd.read_csv('test.csv')
df_tiktok_19 = pd.read_csv('TikTok_songs_2019.csv')
df_tiktok_20 = pd.read_csv('TikTok_songs_2020.csv')
df_tiktok_21 = pd.read_csv('TikTok_songs_2021.csv')

In [64]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [65]:
# Clean up remaining files
!rm data/*
!rm -d data
!rm ./*.csv

# Spotify Top 200 and Viral 50 Charts EDA

In [66]:
# Create a separate column for Year
df_charts['Year']=df_charts['date'].str.slice(stop=4)

# Cast 'Year' as Int
df_charts['Year'] = df_charts['Year'].astype(int)

# Show datatypes 
df_charts.dtypes

#remove ID, trend and streams columns
df_charts.drop(['url', 'trend', 'streams'], axis=1, inplace=True)

df_charts.dropna(inplace=True)

In [67]:
# Number of record per year
df_charts.groupby(by='Year').count()

,title,rank,date,artist,region,chart
Year,,,,,,
2017,4401145,4401145,4401145,4401145,4401145,4401145
2018,4863016,4863016,4863016,4863016,4863016,4863016
2019,5412658,5412658,5412658,5412658,5412658,5412658
2020,5793130,5793130,5793130,5793130,5793130,5793130
2021,5703536,5703536,5703536,5703536,5703536,5703536


In [68]:
# Remore records with year 2017 and 2018
df_charts=df_charts[df_charts['Year'] > 2018]

In [69]:
df_charts.groupby(by='Year').count()

,title,rank,date,artist,region,chart
Year,,,,,,
2019,5412658,5412658,5412658,5412658,5412658,5412658
2020,5793130,5793130,5793130,5793130,5793130,5793130
2021,5703536,5703536,5703536,5703536,5703536,5703536


Top 50 Artists with the most days in number 1 position

In [70]:
query = """SELECT artist, count(date) as 'days as number 1'
FROM df_charts
WHERE rank == 1
GROUP BY artist
ORDER BY count(date) DESC
LIMIT 50
"""

# sql_top_50_artists = ps.sqldf(query, locals())

In [71]:
# sql_top_50_artists


Top 50 songs the longest as number 1

In [72]:
query = """SELECT title, count(date) as 'days as number 1'
FROM df_charts
WHERE rank == 1
GROUP BY title
ORDER BY count(date) DESC
LIMIT 50
"""

# sql_top_50_songs = ps.sqldf(query, locals())

In [73]:
# sql_top_50_songs

# Spotify Audio Feature Dataset EDA

Since the Spotify Top 200 and Viral 50 charts dataset does not come with audio features for the songs, there is no guarantee that we can find datasets that include audio features for all of the songs in the charts. One relevant question to explore is therefore

1. Can we find audio feature data for a suitably "big" fraction of the songs in the Charts dataset?

Additionally, since we'll be combining multiple audio features datasets, there is no guarantee that those datasets share the same features, so

2. What fields do the datasets share?

In [98]:
# First, since we want to find song features for as many of the songs in 'df_charts' as possible, let's start by
# projecting df_charts by song title and artist name, which will uniquely identify each song, and then dropping duplicates
df_charts_songs_artists = df_charts[['title', 'artist']].drop_duplicates()

# Next, let's join the features datasets on song title and artist name and see what percentage of songs in charts_df
# each song features dataset can provide features for

# Clean extranneous characters from artist columns if necessary
df_song_features_1['artists'] = df_song_features_1['artists'].str.replace('[', '').str.replace(']', '').str.replace("'", '')
df_song_features_5['artists'] = df_song_features_5['artists'].str.replace('[', '').str.replace(']', '').str.replace("'", '')
df_song_features_7['artists'] = df_song_features_7['artists'].str.replace(', ', '')
df_song_features_10['artists'] = df_song_features_10['artists'].str.replace(', ', '')

# Project only the necessary columns from each feature dataset and also rename to maintain consistency
features_1_projected = df_song_features_1[['name', 'artists']].rename(columns={'name': 'title', 'artists': 'artist'})
features_2_projected = df_song_features_2[['title', 'artist']]
features_3_projected = df_song_features_3[['Song Name', 'Artist']].rename(columns={'Song Name': 'title', 'Artist': 'artist'})
features_4_projected = df_song_features_4[['track_name', 'artists']].rename(columns={'track_name': 'title', 'artists': 'artist'})
features_5_projected = df_song_features_5[['name', 'artists']].rename(columns={'name': 'title', 'artists': 'artist'})
features_6_projected = df_song_features_6[['track_name', 'artist_name']].rename(columns={'track_name': 'title', 'artist_name': 'artist'})
features_7_projected = df_song_features_7[['t_name0', 'artists']].rename(columns={'t_name0': 'title', 'artists': 'artist'})
features_8_projected = df_song_features_7[['t_name1', 'artists']].rename(columns={'t_name1': 'title', 'artists': 'artist'})
features_9_projected = df_song_features_7[['t_name2', 'artists']].rename(columns={'t_name2': 'title', 'artists': 'artist'})
features_10_projected = df_song_features_10[['t_name0', 'artists']].rename(columns={'t_name0': 'title', 'artists': 'artist'})
features_11_projected = df_song_features_10[['t_name1', 'artists']].rename(columns={'t_name1': 'title', 'artists': 'artist'})
features_12_projected = df_song_features_10[['t_name2', 'artists']].rename(columns={'t_name2': 'title', 'artists': 'artist'})
all_songs_with_features = pd.concat([features_1_projected, features_2_projected, features_3_projected, features_4_projected, 
                                     features_5_projected, features_6_projected, features_7_projected, features_8_projected, 
                                     features_9_projected, features_10_projected, features_11_projected, features_12_projected], ignore_index=True).drop_duplicates()

# Join on song title and artist name
feature_matches = pd.merge(df_charts_songs_artists, all_songs_with_features, on=['title', 'artist'], how='inner')

# Calculate the percentage of songs each features dataset was able to provide song features for
match_percentage_combined = 100 * feature_matches.shape[0] / df_charts_songs_artists.shape[0]
print("We are able to provide audio features for " + str(round(match_percentage_combined,2)) + "% of songs in df_charts,"\
      " for a total of " + str(feature_matches.shape[0]) + " out of " + str(df_charts_songs_artists.shape[0]) + " songs")

We are able to provide audio features for 16.48% of songs in df_charts, for a total of 18950 out of 115014 songs


Lets now see whether we can do any additional cleaning to boost our match percentage. Since there are many obscure songs in df_charts that have, say, reached chart position 200 in small countries like Latvia and Ecuador, let us limit our search to songs that charted highly in the United States, and see if any of those songs were unmatched in the features data

In [99]:
non_matches = pd.merge(df_charts[df_charts['rank'] > 10][df_charts['region'] == 'United States'][['title', 'artist']], 
                                  all_songs_with_features, on=['title', 'artist'], how='left', indicator=True).drop_duplicates()
non_matches = non_matches[non_matches['_merge'] == 'left_only']
non_matches.head()

<ipython-input-99-b56ed4472038>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  non_matches = pd.merge(df_charts[df_charts['rank'] > 10][df_charts['region'] == 'United States'][['title', 'artist']],


,title,artist,_merge
23,Walk Em Down (feat. Roddy Ricch),"Roddy Ricch, NLE Choppa",left_only
53,Social Distancing,Lil Baby,left_only
58,Coffee,"Quinn XCII, Marc E. Bassy",left_only
73,Eleven (feat. Summer Walker),Khalid,left_only
121,Low Down,Lil Baby,left_only


In [76]:
# Does all_songs_with_features contain any of these songs but perhaps with a different format for title and artist?
print(all_songs_with_features[all_songs_with_features['artist'] == 'NLE Choppa, Roddy Ricch'])

                                   title                   artist
991025  Walk Em Down (feat. Roddy Ricch)  NLE Choppa, Roddy Ricch


This suggests that when there is a featured artist, the ordering of the artist names in the features datasets may be the opposite of the ordering in the charts dataset. Lets look at the other row with a featured artist.

In [77]:
print(all_songs_with_features[all_songs_with_features['artist'] == "Khalid, Summer Walker"])

                                title                 artist
1111433  Eleven (feat. Summer Walker)  Khalid, Summer Walker


In this case, the featured artist is not included in the 'artist' column in the charts dataset, but is in the features datasets. In joins so far, we have been joining on 'artist' and 'title' to be sure to uniquely identify each song. But since these songs with featured artists have varying formats in the 'artist' column, but the same format in the 'title' column, and since they are sure to be uniquely identified just by song title, let us try to filter by songs that have "(feat. " in their title, and join the charts and features datasets on 'title'.

In [100]:
charts_featured = df_charts_songs_artists[df_charts_songs_artists['title'].str.contains("\(feat\. ")][['title', 'artist']]
featured_feature_matches = pd.merge(charts_featured, all_songs_with_features, on=['title'], how='inner').drop_duplicates(subset='title')
featured_feature_matches.drop(columns=['artist_x'], inplace=True)
featured_feature_matches.rename(columns={'artist_y': 'artist'}, inplace=True)
feature_matches = pd.concat([feature_matches, featured_feature_matches], ignore_index=True).drop_duplicates()
match_percentage_combined = 100 * feature_matches.shape[0] / df_charts_songs_artists.shape[0]
print("We are able to provide audio features for " + str(round(match_percentage_combined,2)) + "% of songs in df_charts,"\
      " for a total of " + str(feature_matches.shape[0]) + " out of " + str(df_charts_songs_artists.shape[0]) + " songs")

We are able to provide audio features for 17.86% of songs in df_charts, for a total of 20544 out of 115014 songs


Next, lets see what columns each of the features datasets share, and create one unified features dataset.

# TikTok Popular Songs Dataset EDA

In [ ]:
df_tiktok_19.head()

,track_name,artist_name,artist_pop,album,track_pop,danceability,energy,loudness,mode,key,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,duration_ms
0,Shake It,Metro Station,53,Metro Station,68,0.618,0.955,-3.836,1,4,0.0798,0.00221,0.000003,0.486,0.790,150.034,4,179947
1,Chinese New Year,SALES,61,SALES - EP,53,0.744,0.845,-7.422,0,4,0.2530,0.75900,0.232000,0.100,0.749,75.221,4,160000
2,Baby I'm Yours,Breakbot,57,By Your Side,69,0.829,0.792,-3.755,0,2,0.0668,0.72600,0.000006,0.122,0.758,118.050,4,215507
3,The Git Up,Blanco Brown,60,The Git Up,2,0.847,0.678,-8.635,1,9,0.1090,0.06690,0.000000,0.274,0.811,97.984,4,200594
4,Say Hey (I Love You),Michael Franti & Spearhead,58,All Rebel Rockers,0,0.738,0.983,-4.374,0,5,0.0855,0.03800,0.000006,0.183,0.957,92.998,4,235760


In [ ]:
df_tiktok_20.head()

,track_name,artist_name,artist_pop,album,track_pop,danceability,energy,loudness,mode,key,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,duration_ms
0,Say So,Doja Cat,88,Hot Pink,80,0.787,0.673,-4.583,0,11,0.1590,0.26400,0.000003,0.0904,0.779,110.962,4,237893
1,Blinding Lights,The Weeknd,93,After Hours,90,0.514,0.730,-5.934,1,1,0.0598,0.00146,0.000095,0.0897,0.334,171.005,4,200040
2,Supalonely (feat. Gus Dapperton),BENEE,67,Hey u x,63,0.862,0.631,-4.746,1,7,0.0515,0.29100,0.000209,0.1230,0.841,128.978,4,223488
3,Savage,Megan Thee Stallion,82,Suga,70,0.843,0.741,-5.609,1,11,0.3340,0.02520,0.000000,0.0960,0.680,168.983,4,155497
4,Moral of the Story,Ashe,68,Moral of the Story,76,0.572,0.406,-8.624,1,10,0.0427,0.58700,0.000004,0.1020,0.265,119.812,4,201084


In [ ]:
df_tiktok_21.head()

,track_name,artist_name,artist_pop,album,track_pop,danceability,energy,loudness,mode,key,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,duration_ms
0,STAY (with Justin Bieber),The Kid LAROI,83,STAY (with Justin Bieber),89,0.591,0.764,-5.484,1,1,0.0483,0.0383,0.000000,0.1030,0.478,169.928,4,141806
1,Peaches (feat. Daniel Caesar & Giveon),Justin Bieber,90,Justice,86,0.677,0.696,-6.181,1,0,0.1190,0.3210,0.000000,0.4200,0.464,90.030,4,198082
2,edamame (feat. Rich Brian),bbno$,72,edamame (feat. Rich Brian),77,0.815,0.848,-4.704,0,4,0.1150,0.0230,0.000162,0.0265,0.684,106.032,4,133707
3,Here's Your Perfect,Jamie Miller,65,Here's Your Perfect,77,0.558,0.454,-5.335,1,6,0.0551,0.4990,0.000000,0.1460,0.424,113.120,4,158594
4,It's You,Sezairi,57,It's You,73,0.630,0.456,-8.285,1,8,0.0317,0.5450,0.000000,0.1140,0.166,137.915,3,251440


In [ ]:
# Combine the tiktok datasets
data_tiktok = [df_tiktok_19, df_tiktok_20, df_tiktok_21]
df_tiktok_full = pd.concat(data_tiktok)

In [ ]:
# Extract the track name and artist to check for the overlap
df_tiktok_tracks = df_tiktok_full[['track_name', 'artist_name']]

# Check number of rows
df_tiktok_tracks.shape[0]

705

In [ ]:
# Merge with spotify features on track names only to check overlap
df_tiktok_tracks_only = feature_matches.merge(df_tiktok_tracks.drop_duplicates(), left_on = ['title'], right_on=['track_name'], how='inner')

# Check overlap
df_tiktok_tracks_only.shape[0]

601

In [ ]:
# Review rows that didn't match to see if there's more clean-up that can be done 
df_tiktok_full_merge = feature_matches.merge(df_tiktok_tracks.drop_duplicates(), left_on = ['title'], right_on=['track_name'], how='outer', indicator = True)

In [ ]:
df_tiktok_full_merge.head()

,title,artist,track_name,artist_name,_merge
0,Ride It,Regard,Ride It,Regard,both
1,25/8,Bad Bunny,NaN,NaN,left_only
2,Keii,Anuel AA,NaN,NaN,left_only
3,Hei rakas,BEHM,NaN,NaN,left_only
4,GOOBA,6ix9ine,GOOBA,6ix9ine,both


In [ ]:
# Review rows that didn't match to see if there's more clean-up that can be done 
print(df_tiktok_full_merge[df_tiktok_full_merge['_merge'] != 'both'])

            title        artist                              track_name  \
1            25/8     Bad Bunny                                     NaN   
2            Keii      Anuel AA                                     NaN   
3       Hei rakas          BEHM                                     NaN   
5              P2  Lil Uzi Vert                                     NaN   
6      Siren Song         MARUV                                     NaN   
...           ...           ...                                     ...   
19160         NaN           NaN                           Favorite Girl   
19161         NaN           NaN  Love Tonight - David Guetta Remix Edit   
19162         NaN           NaN                           Polaroid Love   
19163         NaN           NaN                                   Panda   
19164         NaN           NaN                             TO THE MOON   

         artist_name      _merge  
1                NaN   left_only  
2                NaN   left_o

In [ ]:
# do a check to see if I convert the track names to lower case, could I get more matches 
df_tiktok_full_merge.dropna(inplace = True)
df_tiktok_full_merge['title_l'] = df_tiktok_full_merge.apply(lambda x : x['title'].lower(), axis =1)
df_tiktok_full_merge['track_name_l'] = df_tiktok_full_merge.apply(lambda x : x['track_name'].lower(), axis =1)

print(df_tiktok_full_merge[(df_tiktok_full_merge['title_l'] == df_tiktok_full_merge['track_name_l']) & (df_tiktok_full_merge['_merge'] != 'both')])

# prints an empty dataframe so the conclusion is that no I can't

Empty DataFrame
Columns: [title, artist, track_name, artist_name, _merge, title_l, track_name_l]
Index: []


In [ ]:
# Create a match column to indicate cases where the tiktok artist name can be found as a substring in the spotify artist column 
df_tiktok_tracks_only['match'] = df_tiktok_tracks_only.apply(lambda x: x['artist_name'].lower().find(x['artist'].lower()), axis=1).ge(0)

# Create a match column to indicate cases where the spotify artist column can be found as a substring in the tiktok artist name 
df_tiktok_tracks_only['match_2'] = df_tiktok_tracks_only.apply(lambda x: x['artist'].lower().find(x['artist_name'].lower()), axis=1).ge(0)

In [ ]:
# review data to see which rows have matched
print(df_tiktok_tracks_only[df_tiktok_tracks_only['match'] |df_tiktok_tracks_only['match_2']])

                                              title              artist  \
0                                           Ride It              Regard   
1                                             GOOBA             6ix9ine   
2                                   Don't Start Now            Dua Lipa   
3                                              Tutu  Camilo, Pedro Capó   
5                  death bed (coffee for your head)  Powfu, beabadoobee   
..                                              ...                 ...   
596                                       Aesthetic                Xilo   
597                        love nwantiti (ah ah ah)                CKay   
598  It Won't Stop (feat. Chris Brown) - EP Version      Sevyn Streeter   
599                   STUPID (feat. Yung Baby Tate)            Ashnikko   
600                                Every Summertime                NIKI   

                                         track_name     artist_name  match  \
0                    

In [ ]:
# review data to see which rows have not matched
print(df_tiktok_tracks_only[(df_tiktok_tracks_only['match'] == False) & (df_tiktok_tracks_only['match_2'] == False)])

                 title                                   artist  \
4                 Tutu                              Grupo Sigma   
8               Say So  Poorchoice, Cam Sanders, Veronica Bravo   
9               Say So                              Masked Wolf   
12             Falling                             Harry Styles   
16              Savage                                     MIST   
..                 ...                                      ...   
511            Asshole                              Denis Leary   
546          Dangerous                                  Roxette   
548       Good Morning                             Trippie Redd   
569  When She Loved Me                          Sarah McLachlan   
580  You Don't Know Me                              Peter Manos   

            track_name          artist_name  match  match_2  
4                 Tutu               Camilo  False    False  
8               Say So             Doja Cat  False    False  
9        

In [ ]:
# check the rows where the artists names aren't an exact match - for visual inspection 
df_tiktok_tracks_only.loc[(df_tiktok_tracks_only['artist'] != df_tiktok_tracks_only['artist_name']) & (df_tiktok_tracks_only['match'] |df_tiktok_tracks_only['match_2'])]

,title,artist,track_name,artist_name,match,match_2
3,Tutu,"Camilo, Pedro Capó",Tutu,Camilo,False,True
5,death bed (coffee for your head),"Powfu, beabadoobee",death bed (coffee for your head),Powfu,False,True
29,Deep End Freestyle,"Sleepy Hallow, Fousheé",Deep End Freestyle,Sleepy Hallow,False,True
41,Lalala,"Y2K, bbno$",Lalala,Y2K,False,True
53,Go Crazy,"Chris Brown, Young Thug",Go Crazy,Chris Brown,False,True
92,Savage Love (Laxed - Siren Beat),"Jawsh 685, Jason Derulo",Savage Love (Laxed - Siren Beat),Jawsh 685,False,True
142,Rodeo,"Lil Nas X, Cardi B",Rodeo,Lil Nas X,False,True
147,Indigo,"88rising, NIKI",Indigo,88rising,False,True
151,Never Ending Story,"Gaten Matarazzo, Gabriella Pizzolo",Never Ending Story,Gaten Matarazzo,False,True
156,Juicy,"Doja Cat, Tyga",Juicy,Doja Cat,False,True


In [ ]:
# Create a dataframe of overlapping songs based on the two matches colums and drop duplicates
df_tiktok_matches_final = df_tiktok_tracks_only.loc[(df_tiktok_tracks_only['match'] |df_tiktok_tracks_only['match_2'])]

# Check the shape of the df 
df_tiktok_matches_final.shape[0]

422

In [ ]:
# spot check an artist to see if the name matching looks appropriate 
df_tiktok_matches_final.loc[(df_tiktok_matches_final['artist_name'] == "Justin Bieber" )|( df_tiktok_matches_final['artist'] == "Justin Bieber" )]

,title,artist,track_name,artist_name,match,match_2
158,Hold On,Justin Bieber,Hold On,Justin Bieber,True,True
174,Ghost,Justin Bieber,Ghost,Justin Bieber,True,True
244,Baby,"Justin Bieber, Ludacris",Baby,Justin Bieber,False,True
366,Peaches (feat. Daniel Caesar & Giveon),Justin Bieber,Peaches (feat. Daniel Caesar & Giveon),Justin Bieber,True,True


In [ ]:
# drop rows that are no longer needed
df_tiktok_matches_final.drop(['match', 'match_2', 'track_name', 'artist_name'], inplace =True, axis =1)

# add a column to indicate tiktok popularity 
df_tiktok_matches_final= df_tiktok_matches_final.assign(tiktok_pop = 1)

<ipython-input-38-d43e3dc5fd77>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tiktok_matches_final.drop(['match', 'match_2', 'track_name', 'artist_name'], inplace =True, axis =1)


In [ ]:
df_tiktok_matches_final.head()

,title,artist,tiktok_pop
0,Ride It,Regard,1
1,GOOBA,6ix9ine,1
2,Don't Start Now,Dua Lipa,1
3,Tutu,"Camilo, Pedro Capó",1
5,death bed (coffee for your head),"Powfu, beabadoobee",1


In [ ]:
df_tiktok_matches_final.groupby(['tiktok_pop']).count()

,title,artist
tiktok_pop,,
1,422,422


# Genius Lyrics Dataset Creation and EDA